# Automated ML

Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import requests
import json
import logging
import joblib
import pandas as pd
from sklearn.model_selection import train_test_split

import azureml.core
from azureml.core.workspace import Workspace
from azureml.core.experiment import Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.core.compute_target import ComputeTargetException
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.widgets import RunDetails
from azureml.core.model import Model, InferenceConfig
from azureml.core.webservice import AciWebservice

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

## Initialize Workspace

Initialize a workspace object from persisted configuration. Make sure the config file is present at ./config.json

In [ ]:
ws = Workspace.from_config()

print('Workspace name: ' + ws.name,
      'Azure region: ' + ws.location,
      'Subscription id: ' + ws.subscription_id,
      'Resource group: ' + ws.resource_group, sep='\n')


## Create an Azure ML experiment
Let's create an experiment named "automl-experiment-heart-failure-classification" and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the source directory for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the source directory would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the source_directory of the step.


In [ ]:
experiment_name = 'automl-experiment-heart-failure-classification'
project_folder = './classification-project'

experiment = Experiment(ws, experiment_name)
experiment

## Create or Attach an AmlCompute cluster
We will need to create a compute target for our AutoML run. We will use `vm_size = Standard_D2_V2` in our provisioning configuration and select `max_nodes` to be no greater than 4.

In [ ]:
aml_compute_cluster_name = "cpu-cluster"

# Verify that cluster doesn't exist already
try:
    aml_compute = ComputeTarget(workspace=ws, name=aml_compute_cluster_name)
    print("Found existing cluster, use it.")

except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2",
                                                           min_nodes=1,
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(workspace=ws,
                                       name=aml_compute_cluster_name,
                                       provisioning_configuration=compute_config)
    
aml_compute.wait_for_completion(show_output=True)

In [ ]:
compute_targets = ws.compute_targets

for i, key in enumerate(compute_targets):
    print(f"{i+1}. Compute target\n\tname: {compute_targets[key].name}\n\tType: {compute_targets[key].type}\n")
    

## Dataset

### Overview

This dataset contains the medical records of 299 patients who had heart failure, collected during their follow-up period, where each patient profile has 13 clinical features.

The 12 clinical input features and the target feature are:

- age: age of the patient (years)
- anaemia: decrease of red blood cells or hemoglobin (boolean)
- high blood pressure: if the patient has hypertension (boolean)
- creatinine phosphokinase (CPK): level of the CPK enzyme in the blood (mcg/L)
- diabetes: if the patient has diabetes (boolean)
- ejection fraction: percentage of blood leaving the heart at each contraction (percentage)
- platelets: platelets in the blood (kiloplatelets/mL)
- sex: woman or man (binary)
- serum creatinine: level of serum creatinine in the blood (mg/dL)
- serum sodium: level of serum sodium in the blood (mEq/L)
- smoking: if the patient smokes or not (boolean)
- time: follow-up period (days)
- [target] death event: if the patient deceased during the follow-up period (boolean)

The task we are concerned with is to predict if the patient deceased during the follow-up period. We will be using `DEATH_EVENT` column as the target and since this is a boolean variable, the task at hand is Binary Classification. 

In the cell below, we write code to access the data that we will be using in this project.

We will try to load the dataset from the Workspace. If it isn't found because it was deleted, it can be recreated with the link that has the CSV

Make sure the `key` is the same name as the dataset that is uploaded. Also we provide a matching description. If it is hard to find or unknown, loop over the `ws.datasets.keys()` and `print()` them. 


In [ ]:
ws.datasets.keys()

In [ ]:
found = False
key = "Health-Failure"
description_text = "Health Failure dataset for mortality prediction"

dataset_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv"

if key in ws.datasets.keys():
    dataset = ws.datasets[key]
    print("The Dataset was found!")
else:
    # Create AML Dataset and register it into Workspace
    dataset = Dataset.Tabular.from_delimited_files(dataset_url)
    #Register Dataset in Workspace
    dataset = dataset.register(workspace=ws,
                               name=key,
                               description=description_text)

df = dataset.to_pandas_dataframe()

In [ ]:
df.describe()

In [ ]:
# Split the dataset into training and testing datasets
train_df, test_df = train_test_split(df, test_size=0.2)

# Save training data to csv file
train_df.to_csv("./train_data.csv", index=False)

# Read saved training data and create a dataset in Azure ML
data_store = ws.get_default_datastore()
data_store.upload(src_dir="./", target_path="training_data")
train_ds = TabularDatasetFactory.from_delimited_files(path=[(data_store, 'training_data/train_data.csv')])


## Review the Dataset Result

You can peek the result of a TabularDataset at any range using `skip(i)` and `take(j).to_pandas_dataframe()`. Doing so evaluates only `j` records for all the steps in the `TabularDataset`, which makes it fast even against large datasets.

`TabularDataset` objects are composed of a list of transformation steps (optional).

In [ ]:
train_ds.take(5).to_pandas_dataframe()

## AutoML Configuration

As mentioned above in the dataset section we are dealing with a binary classification. Therefore the argument `task` is set to `classification` and the since we are predicting `DEATH_EVENT` we need to set `label_column_name="DEATH_EVENT"`

To help manage child runs and when they can be performed, we recommend you create a dedicated cluster per experiment, and match the number of `max_concurrent_iterations` of your experiment to the number of nodes in the cluster. This way, you use all the nodes of the cluster at the same time with the number of concurrent child runs/iterations you want.

Configure `max_concurrent_iterations` in your `AutoMLConfig` object. If it is not configured, then by default only one concurrent child run/iteration is allowed per experiment.

Besides other arguments that are self-explanatory, to automate Feature engineering AzureML enables this through `featurization` that needs to be set to `True`. This way features that best characterize the patterns in the data are selected to create predictive models.

In [ ]:
# AutoMl settings
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 4,
    "primary_metric": "AUC_weighted",
    "enable_early_stopping": True,
    "verbosity": logging.INFO
}

# AutoMl config
automl_config = AutoMLConfig(compute_target=aml_compute,
                             task="classification",
                             training_data=train_ds,
                             label_column_name="DEATH_EVENT",
                             n_cross_validations=5,
                             featurization="auto",
                             path=project_folder,
                             debug_log = "automl_errors.log",
                             **automl_settings                             
                            )

In [ ]:
# Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

## Run Details

In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

## Best Model

In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
# Retrieve your best automl model.
best_run, best_model = remote_run.get_output()
best_run_metrics = best_run.get_metrics()

In [ ]:
best_model

In [ ]:
best_run

In [ ]:
best_run_metrics

In [ ]:
best_run.get_details()

In [ ]:
best_run.get_properties()

In [ ]:
# Save the best model
joblib.dump(best_model, filename="./best_automl_model.joblib")

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
model = Model.register(workspace=ws,
                       model_name="heart_failure_pred_model", 
                       model_path="./best_automl_model.joblib",
                       description="Best AutoML model"
                      )

In [ ]:
env = best_run.get_environment()

inf_config = InferenceConfig(environment=env,
                             entry_script='./score.py')

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1,
                                                       memory_gb=1,
                                                       auth_enabled=False,
                                                       enable_app_insights=True)

service = Model.deploy(workspace=ws, 
                       name="automl-service",
                       models=[model],
                       inference_config=inf_config,
                       deployment_config=deployment_config)

service.wait_for_deployment(show_output=True)

Chech the state of the deployed service and get its URIs 

In [ ]:
print(f"Service State: {service.state}\n")
print(f"Scoring URI: {service.scoring_uri}\n")
print(f"Swagger URI: {service.swagger_uri}\n")

In the cell below, send a request to the web service you deployed to test it.

In [ ]:
# URL for the web service
scoring_uri = service.scoring_uri

# If the service is authenticated, set the key or token
#key = ""

# 3 sets of data to score, so we get two results back
data_df = test_df.sample(n=3)
labels = data_df.pop('DEATH_EVENT')


# Convert to JSON string
input_data = json.dumps({"data": data_df.to_dict(orient='records')})
with open("input_data.json", 'w') as _f:
    _f.write(input_data)

print(input_data)

In [ ]:
# Set the content type
headers = {"Content-Type": "application/json"}

# If authentication is enabled, set the authorization header
#headers["Authorization"] = f"Bearer {key}"

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())

In [ ]:
print(f"Predictions from Service: {resp.json()}\n")
print(f"Data Labels: {labels.tolist()}")

In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.get_logs())

In [ ]:
service.delete()